In [36]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import random
import os
import json
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)

In [83]:
train_dataset['train'].features.keys()

dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])

In [65]:
train_dataset = load_from_disk('/opt/ml/input/data/train_dataset')
test_dataset = load_from_disk('/opt/ml/input/data/test_dataset')

In [39]:
train_dataset['train']

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})

In [40]:
corpus = list(set([example['context'] for example in train_dataset['train']]))
len(corpus)

3340

In [17]:
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 15.0kB/s]
Downloading: 100%|██████████| 625/625 [00:00<00:00, 291kB/s]
Downloading: 100%|██████████| 996k/996k [00:01<00:00, 736kB/s]  
Downloading: 100%|██████████| 1.96M/1.96M [00:01<00:00, 1.42MB/s]


In [44]:
# Use subset (128 examples) of original training dataset 
sample_idx = np.random.choice(range(len(train_dataset['train'])), 128)
training_dataset = train_dataset['train'][sample_idx]

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

In [57]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

In [58]:
train_sampler

In [54]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)
 
    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]
 
      return pooled_output
 
 
def train(args, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)
 
  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
 
  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
 
  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
 
    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
 
      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2]
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)
 
 
      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_ouatputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)
 
      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')
 
      sim_scores = F.log_softmax(sim_scores, dim=1)
 
      loss = F.nll_loss(sim_scores, targets)
 
      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()
 
 
    
  return p_model, q_model

In [55]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)

In [56]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Downloading: 100%|██████████| 714M/714M [00:11<00:00, 60.6MB/s] 
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were

In [11]:
data_path = "/opt/ml/input/data/"
context_path = "wikipedia_documents.json"

In [12]:
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

In [61]:
search_corpus = list(dict.fromkeys([v["text"] for v in wiki.values()]))  # set 은 매번 순서가 바뀌므로
print(f"Lengths of unique contexts : {len(contexts)}")
ids = list(range(len(contexts)))

Lengths of unique contexts : 56737


In [62]:
eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)


Iteration: 100%|██████████| 7093/7093 [09:41<00:00, 12.19it/s]


(56737, 768)

In [66]:
np.random.seed(1)

sample_idx = np.random.choice(range(len(train_dataset['validation'])), 5)
query = train_dataset['validation'][sample_idx]['question']
ground_truth = train_dataset['validation'][sample_idx]['context']

query

['로마의 공성무기에 대한 기록을 남긴 사람은?',
 '전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?',
 '외국어영화상 위원회에서 최종 후보 다섯 편을 추리는 방법은?',
 '교황의 문장에서 교차한 금빛 열쇠와 은빛 열쇠가 뜻하는 바는?',
 '왕필과 함께 대표적인 현학자로 불리며 《장자》에 주석을 단 사람은?']

In [84]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

In [68]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [69]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[54667, 19803, 55063,  ..., 38335, 23533, 38653],
        [54667, 19803, 55063,  ..., 38335, 23533, 38653],
        [54667, 19803, 55063,  ..., 38335, 23533, 38653],
        [54667, 19803, 55063,  ..., 38335, 23533, 38653],
        [54667, 19803, 55063,  ..., 38335, 23533, 38653]], device='cuda:0')
--- 0.010508298873901367 seconds ---


In [88]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 로마의 공성무기에 대한 기록을 남긴 사람은? 

[Ground truth passage]
후기 로마군\n로마 군단 목록\n \n로마 제국 말기에, 군단의 수는 늘어났고 로마군은 확장됐다. 일반적으로 인용된 서류상 병력들보다 수가 작아졌다는 증거는 존재하지만, 사두정치 이전에 군단이 형태에 변화가 있었는지를 나타내는 증거는 없다. 군단의 최종 형태는 디오클레티아누스와 사두정치 때 만든 정예 부대 레기오네스 팔라티나이에서 기원했다. 이 부대는 옛 군단의 5,000명 규모보다는 기병을 포함해, 대략 1,000명 규모의 보병 부대이다. 초창기 레기오네스 팔라티나이는 란키아리이, 요비아니, 에르쿨리아니, 디비텐세스 등의 부대가 있었다.\n\n4세기에 콘스탄티누스 2세 제위 시기에 시작된 과정으로, 수 많은 수의 새롭고 소규모의 군단들이 창설됐다. 정예 부대 팔라티니외에도, 아욱실리아 팔라티나라는 보조병들과 더불어, 코미타텐세스, 프세우도코미타텐세스라는 군단들이 제정 말기 로마에 보병들을 제공했다. 노티티아 디그니타툼은 야전 부대들인 팔라티나이 군단 25개, 코미타텐세스 군단 70개, 프세우도코미타텐세스 군단 47개, 아욱실리아 팔라티나 111개와 국경 수비 부대 47개 군단을 언급한다. 노티티아 디그니타툼에서 발견된 호노리아니, 그라티아넨세스 같은 군단 명칭들은 새로운 군단들을 만드는 과정이 단발성의 사건이라기보다는 4세기 전반에 지속되었다는 것을 시사한다. 이 명칭들은 또한 많은 새로운 군단들이 벡실라티오네스 또는 옛 군단에서 만들어졌음을 말해준다. 그외에, 벡실라티오네스 팔라티니 24개, 벡실라티오네스 코미타텐세스 73개가 있었고, 그 밖에 부대들이 동방 지역의 리미타네이에 305개, 서방 지역의 리미타네이에 181개가 있었다. 제정 초기 군단과 6세기 이후 군단 사이에 명백한 직접적인 연속성에 대한 드문 예는 기원전 43년에 창설된 마케도니카 제5군단으로, 노티티아 디그니타툼에 마케도니카 5군단이라는 이름의 코미타텐세 군단으로 기록되었